In [ ]:
!pip install jschon

In [ ]:
!pip install js2py

In [ ]:
from jschon import create_catalog,JSON, JSONSchema
import json,os
import pathlib
import inspect
import  js2py


In [ ]:
create_catalog('2019-09')


In [ ]:
keywords_to_avoid = ['not','anyOf','oneOf','allOf','patternPropreties']
def extract_errors_map(error:dict, error_map:dict):
    # print(error)
    # if there's errors in subschema treat them since they are the origin of the error  
    if 'errors' in error:
        for e in error['errors']:
            extract_errors_map(e, error_map)
    else : 
        keywordLocation = str(error['keywordLocation'])
        key = keywordLocation.split('/')[-1]
        
        if error["valid"] == True :
            if keywordLocation.__contains__("not"):
                if error_map.get(key) is None : 
                    error_map[key]=[error['error'] if 'error' in error else '']
                else : 
                    error_map[key].append(error['error'] if 'error' in error else '') 
                
            pass 
        else :      
            if key and  'error' in error and key not in keywords_to_avoid:
                if error_map.get(key) is None : 
                    error_map[key]=[error['error'] if 'error' in error else '']
                else : 
                    error_map[key].append(error['error'] if 'error' in error else '')
    
    # print("there's still error")
   

def extract_errors(errors): 
    error_map= {}
    for error in errors : 
        extract_errors_map(error,error_map)
    return list(error_map.keys())

In [268]:
import json
import subprocess
def migrate(schema):
    try:
        # Convertir le schéma JSON en chaîne JSON
        schema_json_str = json.dumps(schema)
        # Appeler le script TypeScript avec le schéma JSON en tant qu'argument
        command = ['ts-node', '-e', f'require("./migrate").migrate({schema_json_str})']
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=True, text=True)
        stdout, stderr = process.communicate()
        # Vérifier si une erreur s'est produite pendant l'exécution du script TypeScript
        if stderr:
            raise RuntimeError(f"Erreur lors de l'exécution du script TypeScript : {stderr}")

        # Analyser la sortie JSON du résultat
        result = json.loads(stdout)
        return result

    except Exception as e:
        # print(str(e))
        print(f"Une erreur s'est produite : {e}")
        return None

In [269]:

def validate_instance(schema_dir:str,instance_dir): 
    with open(schema_dir,'r',encoding='utf-8') as f : 
        schema= json.load(f)
    print(schema)
    translated_schema=migrate(schema)
    print(translated_schema)
    # demo_schema = JSONSchema(translated_schema)
    # # print(demo_schema)
    # with open (instance_dir,'r',encoding='utf-8') as f : 
    #     instance=json.load(f)
        
    # result = demo_schema.evaluate(
    #     JSON(instance)
    # )
    # return result 
def get_errors_validation(schema_dir:str,instance_dir:str): 
    result = validate_instance(schema_dir,instance_dir)
    # print(result.output('detailed'))# if errors in result.output('detailed')
    if(result.output("verbose")['valid']==False or "errors" in result.output("verbose")): 
        errors = extract_errors(result.output('verbose')['errors'])
    else : 
        errors=[]
    return errors

In [270]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))
schema_path = src_file_path+'/dataset/8/schema.json'
instance_path = src_file_path+'/dataset/8/instance.json'
# output = validate_instance(schema_path,instance_path).output('verbose')
# with open(src_file_path+'/dataset/8/errors.json','w') as f: 
#     json.dump(output,f)
validate_instance(schema_path,instance_path)

{'$schema': 'http://json-schema.org/draft-04/schema#', 'id': 'https://raw.githubusercontent.com/AlekseyBuzmakov/FCAPS/master/FCAPS/schemas/graph.json', 'title': 'Directed Graph JSON', 'description': 'Json file gives a description of a directed graph', 'type': 'array', 'items': [{'type': 'object', 'title': 'General Grap Info', 'description': 'An object contains general info about the graph', 'properties': {'NodesCount': {'title': 'Number of Nodes', 'description': 'The total number of nodes in the graph', 'type': 'integer', 'minimum': 0}, 'ArcsCount': {'title': 'Number of Arcs', 'description': 'The total number of arcs in the graph', 'type': 'integer', 'minimum': 0}}}, {'title': 'Nodes Info', 'description': 'All nodes are enumerated in this object', 'type': 'object', 'properties': {'Nodes': {'title': 'List of Nodes', 'description': 'The nodes of the graph are given within this array', 'type': 'array', 'items': {}}}, 'required': ['Nodes']}, {'title': 'Arcs Info', 'description': 'All arcs 

In [ ]:
src_file_path = os.path.dirname(os.path.abspath("__file__"))

# print(src_file_path)
dataset_dir =  src_file_path+'/dataset'
for dir in os.listdir(dataset_dir) : 
    if dir=='instances' or dir=='schemas' : 
        continue 
    print(f"Treating schema : {dir}.json")
    schema_dir = f'{dataset_dir}/{dir}/schema.json'
    instance_dir=f'{dataset_dir}/{dir}/instance.json'
    errors=get_errors_validation(schema_dir,instance_dir)
    print(errors)